In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import src.urban_diff_helper as ud_helper
import pandas as pd
from scipy.stats import wilcoxon,ttest_rel,kendalltau, spearmanr

In [3]:
import numpy as np

In [4]:
from IPython.display import display

# load geometries

In [5]:
zms, zms_agebs, zms_tvor, zms_grids, mg_mappings = ud_helper.load_geoms()

loading zms
loading agebs
loading voronoi
loading existing mexico tower voronoi file: data/mex_tw_intpl/voronoi.geojson
==================== loading grids False False 2019-11-04 14:42:51.432993
=====computing mpa_grids
reading existing grids
==================== loading grids False True 2019-11-04 14:44:37.708043
loading existing file data/mexico/geography-socioeconomics/2010MetropolitanAreas/grids_500_whole_urb.geojson.gz
==================== loading grids True False 2019-11-04 14:44:42.231241
loading existing file data/mexico/geography-socioeconomics/2010MetropolitanAreas/grids_500_perMun_uNr.geojson.gz
==================== loading grids True True 2019-11-04 14:45:23.732792
loading existing file data/mexico/geography-socioeconomics/2010MetropolitanAreas/grids_500_perMun_urb.geojson.gz


# load interpolation and interpolate avg

In [6]:
avg_tw, avg_a, avg_g, avg_idw = ud_helper.interpolation(zms_grids)

loading cached tw average stats/stat_tw_dow_aver_hr_uniq_user-out+in.pickle
to_map_agebs loading existing file data/mex_tw_intpl/tower_to_mpa_agebs_by_area.csv
to_map_agebs loading existing file data/mex_tw_intpl/tower_to_mpa_agebs_by_pop.csv
to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g500_whole_uNr_area.csv
to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g500_whole_urb_area.csv
to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g500_perMun_uNr_area.csv
to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g500_perMun_urb_area.csv
to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g500_whole_uNr_pop.csv
to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g500_whole_urb_pop.csv
to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g500_perMun_uNr_pop.csv
to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g500_perMun_urb_pop.csv
interpolate_idw loading existing file

# statistics test

In [7]:
def sig_lvl(x):
    if x==-1 or x==0:
        return '-'
    if x <0.001:
        return '***'
    if x<0.01:
        return '**'
    if x<0.05:
        return '*'
    return ''

In [8]:
def statistics_test(test_func, stat_df, clean_table=False, larger_than=0.0):

    n_cols = stat_df.shape[1]
    p_values = np.zeros((n_cols, n_cols))
    test_measures = np.ones((n_cols, n_cols))
    for i in range(n_cols):
        for j in range(i + 1, n_cols):
            m, p = test_func(stat_df.iloc[:, i], stat_df.iloc[:, j])
            p_values[i, j] = p_values[j, i] = p
            test_measures[i, j] = test_measures[j, i] = m

    
    p_values = pd.DataFrame(p_values,
                          index=stat_df.columns,
                          columns=stat_df.columns)
    test_measures[test_measures<=larger_than] = 0
    test_measures = pd.DataFrame(test_measures,
                          index=stat_df.columns,
                          columns=stat_df.columns)
    
    if clean_table:
        p_values = p_values.applymap(sig_lvl)
#         test_measures = test_measures.applymap(lambda x: f'{x:.3f}')
    return p_values, test_measures

wilcoxon_test = lambda stat_df, clean_table, larger_than: statistics_test(wilcoxon, stat_df, clean_table, larger_than)
paired_t_test = lambda stat_df, clean_table, larger_than: statistics_test(ttest_rel, stat_df, clean_table, larger_than)
ken_tau_corr = lambda stat_df, clean_table, larger_than: statistics_test(kendalltau, stat_df, clean_table, larger_than)
spe_rho_corr = lambda stat_df, clean_table, larger_than: statistics_test(spearmanr, stat_df, clean_table, larger_than)

# draw function

In [9]:
import plotly.offline as offline
offline.init_notebook_mode(connected=True)
import plotly.graph_objects as go

In [10]:
def draw_scatter(data_df, mode='lines+markers',height=600,margin_r=100):
    
    data = [
        go.Scatter(x='zm' + data_df.index.astype(str),
                   y=data_df[intpl_type].tolist(),
                   name=str(intpl_type),
                   mode='lines+markers'
                  ) for intpl_type in data_df
    ]
    fig = go.Figure()
    for trace in data:
        fig.add_trace(trace)
    fig.update_layout(height=height,margin=dict(l=10, r=margin_r, t=10, b=10),)

    offline.iplot(fig)

In [11]:
import matplotlib.pyplot as plt
from matplotlib import colors

def background_gradient(s, m=None, M=None,  cmap='Oranges', low=0, high=0.2):

    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]


In [12]:
swap = lambda df, i, j: df.swaplevel(i,j).T.swaplevel(i,j).sort_index(level=range(len(df.columns.levels))).T.sort_index(level=range(len(df.columns.levels)))

In [13]:
from IPython.core.display import HTML 

In [14]:
def grad_bg(df, cmap='Oranges', mini=None, maxi=None, high=0, mask_tril=False):
    if mini is None:
        mini = df.min().min()
    if maxi is None:
        maxi = df.max().max()
    if mask_tril:
        df = df.mask(np.tril(np.ones(df.shape, dtype=np.bool_),-1))
    style= df.style.apply(background_gradient, m=mini, M=maxi, cmap=cmap, high=high).format("{:.3f}").set_table_attributes("style='display:inline'")
    return HTML(style.render().replace('nan',''))

# urban dilatation index

In [20]:
# compute
print('computing dv_a')
dv_a = {}
for by, avg in avg_a.items():
    dv_a[by] = ud_helper.dilatation.dv_for_mpa_ageb(avg, zms, zms_agebs)

print('computing dv_g')
dv_g = {}
for (side, by, per_mun), avg in avg_g.items():
    zms_g = zms_grids[(side, per_mun)]
    dv_g[(side, by, per_mun)] = ud_helper.dilatation.dv_for_mpa_grids(avg, zms, zms_g)

print('computing dv_idw')
dv_idw = {}
for key, avg in avg_idw.items():
    zms_g = zms_grids[key]
    dv_idw[key] = ud_helper.dilatation.dv_for_mpa_grids(avg, zms, zms_g)

computing dv_a
computing dv_g
computing dv_idw


In [437]:
# flatten the dictionary of stats
var = 'dilatation coefficient'
a = {('ageb', 'area'):dv_a['area'][var], ('ageb', 'pop') : dv_a['pop'][var]}
g= {(f'g{side//1000}', by, ):dv[var] for (side, by, per_mun), dv in dv_g.items() if not per_mun}
idw= {(f'g{side//1000}','_idw') :dv[var] for (side, per_mun), dv in dv_idw.items() if not per_mun}
dila_coef = pd.DataFrame({**a, **g, **idw})

In [278]:
# Visualization
sort_col = ud_helper.zms_sort_cols[0]
dila_coef = dila_coef.loc[zms.sort_values(sort_col).index]

# sort_col = dila_coef.columns[0]
# dila_coef=dila_coef.sort_values(sort_col)
# print(sort_col)

draw_scatter(dila_coef, height=300)
draw_scatter(dila_coef.std(axis=1).to_frame(), height=300, margin_r=135)

In [460]:
64*59

3776

In [456]:
# even though (ageb, pop) and (g1 pop) has similar values, 
# but the std could also be small.
dila_wil_ps, _ = wilcoxon_test(dila_coef, clean_table=False)
grad_bg(dila_wil_ps, cmap='PuBu',high=.2)
# dila_wil_ps

In [470]:
dila_ken_ps, dila_ken_tau = ken_tau_corr(
    dila_coef, clean_table=False, larger_than=0.8)
print()
grad_bg(dila_ken_ps, cmap='PuBu',high=.2)

In [471]:
grad_bg(swap(dila_ken_tau,0,1))

In [472]:
grad_bg(swap(swap(dila_ken_tau,0,1),0,1))

# hotspots

In [15]:
# compute hot stats
hotspot_type = 'loubar'
n_hs_a = {}
comp_coef_a = {}
for by in ['area', 'pop']:
    for per_mun in [False, True]:
        for urb_only in [False, True]:
            key = (by, per_mun, urb_only)
            print(key, end=' ')
            n, cc = ud_helper.ftr_hs.hs_stats_ageb(
                avg_a[by], zms, zms_agebs, mg_mappings, per_mun, urb_only, hotspot_type)
            n_hs_a[key] = n
            comp_coef_a[key] = cc

n_hs_g = {}
comp_coef_g = {}
for key, avg in avg_g.items():
    print(key, end=' ')
    by, per_mun, urb_only = key
    zms_g = zms_grids[(per_mun, urb_only)]
    n, cc = ud_helper.ftr_hs.hs_stats_grid(
        avg, zms, zms_g, per_mun, hotspot_type)
    n_hs_g[key] = n
    comp_coef_g[key] = cc

n_hs_idw = {}
comp_coef_idw = {}
for key, avg in avg_idw.items():
    print(key, end=' ')
    per_mun, urb_only = key
    zms_g = zms_grids[key]
    n, cc = ud_helper.ftr_hs.hs_stats_grid(
        avg, zms, zms_g, per_mun, hotspot_type)
    n_hs_idw[key] = n
    comp_coef_idw[key] = cc


n_hs_tw = {}
comp_coef_tw = {}
for per_mun in [False, True]:
    for urb_only in [False, True]:
        key = (per_mun, urb_only)
        print(key, end=' ')
        n, cc = ud_helper.ftr_hs.hs_stats_vor(avg_tw, zms, per_mun, urb_only, hotspot_type)
        n_hs_tw[key] = n
        comp_coef_tw[key] = cc

('area', False, False) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
('area', False, True) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
('area', True, False) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
('area', True, True) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
('pop', False, False) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 

21 (`14044`=1 no hotspot) 22 23 24 (`15018`=1 no hotspot) (`15062`=2 using `average`) (`15118`=2 using `average`) 25 (`16088`=2 using `average`) 26 27 (`11023`=2 using `average`) 28 (`17020`=1 no hotspot) 29 (`17002`=1 no hotspot) 30 (`18008`=2 using `average`) 31 (`19010`=1 no hotspot) (`19045`=2 using `average`) 32 (`20107`=1 no hotspot) (`20115`=1 no hotspot) (`20157`=1 no hotspot) (`20227`=1 no hotspot) (`20350`=1 no hotspot) (`20553`=1 no hotspot) 33 (`20515`=2 using `average`) 34 (`21015`=2 using `average`) (`21034`=2 using `average`) (`21041`=2 using `average`) (`21074`=1 no hotspot) (`21125`=1 no hotspot) (`21136`=1 no hotspot) (`29025`=1 no hotspot) (`29041`=2 using `average`) (`29042`=1 no hotspot) (`29044`=1 no hotspot) 35 36 (`22008`=1 no hotspot) (`22011`=1 no hotspot) 37 (`23003`=2 using `average`) 38 39 (`24011`=2 using `average`) 40 41 (`27013`=1 no hotspot) 42 43 44 45 46 (`29002`=1 no hotspot) (`29005`=2 using `average`) (`29010`=1 no hotspot) (`29018`=1 no hotspot) (

In [16]:
from src.urban_diff_helper import ADMIN_STR

In [17]:

a = {(ADMIN_STR(per_mun, urb_only), 'ageb', by, ):cc for (by, per_mun, urb_only), cc in comp_coef_a.items() }
g = {(ADMIN_STR(per_mun, urb_only), f'g.5km',by,): cc for (by,per_mun, urb_only), cc in comp_coef_g.items()}
idw = {(ADMIN_STR(per_mun, urb_only), f'g.5km','_idw10'): cc for (per_mun, urb_only), cc in comp_coef_idw.items()}
vor = {(ADMIN_STR(per_mun, urb_only), f'vor',''): cc for (per_mun, urb_only), cc in comp_coef_tw.items()}

comp_coef = pd.DataFrame({**a, **g, ** idw, **vor})

In [18]:

a = {(ADMIN_STR(per_mun, urb_only), 'ageb', by, ):cc for (by, per_mun, urb_only), cc in n_hs_a.items() }
g = {(ADMIN_STR(per_mun, urb_only), f'g.5km',by,): cc for (by,per_mun, urb_only), cc in n_hs_g.items()}
idw = {(ADMIN_STR(per_mun, urb_only), f'g.5km','_idw10', ): cc for (per_mun, urb_only), cc in n_hs_idw.items()}
vor = {(ADMIN_STR(per_mun, urb_only), f'vor',''): cc for (per_mun, urb_only), cc in n_hs_tw.items()}

Mean_number_hot = pd.DataFrame({**a, **g, **idw, **vor})

## comp_coef

In [19]:
comp_ken_ps, comp_ken_tau = ken_tau_corr(comp_coef, clean_table=False,larger_than=.0)
comp_spr_ps, comp_spr_rho = spe_rho_corr(comp_coef, clean_table=False, larger_than=.0)

In [40]:

swap_df = comp_ken_tau

display(grad_bg(swap_df.T.sort_index(level=0).T.sort_index(level=0), mask_tril=True))

# swap_df = swap(swap_df, 0,2)
# # display(grad_bg(swap_df))
# swap_df = swap(swap_df, 1,2)
# display(grad_bg(swap_df))

/home/Jiahui/miniconda3/envs/mob2crime/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning:

invalid value encountered in less



In [236]:


sort_col = ud_helper.zms_sort_cols[3]
comp_coef = comp_coef.loc[zms.sort_values(sort_col).index]
sort_col = comp_coef.columns[0]
comp_coef=comp_coef.sort_values(sort_col)
print(sort_col)

draw_scatter(comp_coef, height=300)
draw_scatter(comp_coef.std(axis=1).to_frame(), height=300, margin_r=175)

('whole_uNr', 'ageb', 'area')


## N hotspot

In [20]:
nhs_spr_ps, nhs_spr_rho = spe_rho_corr(Mean_number_hot, clean_table=False,larger_than=0.0)
nhs_ken_ps, nhs_ken_tau = ken_tau_corr(Mean_number_hot, clean_table=False,larger_than=0.0)


In [379]:
# display(grad_bg(nhs_ken_tau.T.sort_index(level=0).T.sort_index(level=0),mini=0))

# swap_df = swap(nhs_ken_tau, 0,2)
# display(grad_bg(swap_df))

# swap_df = swap(swap_df, 1,2)
# display(grad_bg(swap_df))
# swap_df = swap(swap_df, 2,0)
# display(grad_bg(swap_df))

In [288]:

sort_col = Mean_number_hot.columns[0]
Mean_number_hot=Mean_number_hot.sort_values(sort_col)
print(sort_col)

draw_scatter(Mean_number_hot)

('ageb', nan, 'area', 'zm')


## display table

In [21]:
admin_strs = [ADMIN_STR(m,u) for m in [False, True] for u in [False,True]]

In [36]:
def print_table(astr, measurements):
    print(astr.replace('_','\n'))
    for index_name, index_corr in measurements:
        tbl = index_corr.T.sort_index(level=0).T.sort_index(level=0).loc[astr, astr].copy()
        tbl.columns =['<br/>'.join(col).strip() for col in tbl.columns.values]
        tbl.index =['<br/>'.join(col).strip() for col in tbl.index.values]
        tbl.index.name=index_name
        dis_tbl = grad_bg(tbl, mask_tril=True, mini=0)
        display(dis_tbl)

In [37]:
i=3
astr=admin_strs[i]
measurements = [
    ('Comp Coef', comp_ken_tau), 
    ('Avg N HS', nhs_ken_tau)
]
print_table(astr, measurements)

PerMun
UrbanOnly


,agebarea,agebpop,g.5km_idw10,g.5kmarea,g.5kmpop,vor
Comp Coef,,,,,,
agebarea,1.000,0.619,0.607,0.755,0.576,0.323
agebpop,,1.000,0.640,0.640,0.735,0.396
g.5km_idw10,,,1.000,0.736,0.693,0.443
g.5kmarea,,,,1.000,0.721,0.379
g.5kmpop,,,,,1.000,0.390
vor,,,,,,1.000


,agebarea,agebpop,g.5km_idw10,g.5kmarea,g.5kmpop,vor
Avg N HS,,,,,,
agebarea,1.000,0.780,0.586,0.648,0.727,0.641
agebpop,,1.000,0.525,0.604,0.704,0.663
g.5km_idw10,,,1.000,0.676,0.698,0.454
g.5kmarea,,,,1.000,0.801,0.451
g.5kmpop,,,,,1.000,0.548
vor,,,,,,1.000


In [38]:
# i = 0
astr = admin_strs[i]
measurements = [
    ('Comp Coef', comp_spr_rho), 
    ('Avg N HS', nhs_spr_rho)
]
print_table(astr, measurements)

PerMun
UrbanOnly


,agebarea,agebpop,g.5km_idw10,g.5kmarea,g.5kmpop,vor
Comp Coef,,,,,,
agebarea,1.000,0.751,0.813,0.915,0.742,0.413
agebpop,,1.000,0.814,0.776,0.863,0.521
g.5km_idw10,,,1.000,0.903,0.845,0.640
g.5kmarea,,,,1.000,0.854,0.503
g.5kmpop,,,,,1.000,0.543
vor,,,,,,1.000


,agebarea,agebpop,g.5km_idw10,g.5kmarea,g.5kmpop,vor
Avg N HS,,,,,,
agebarea,1.000,0.926,0.782,0.826,0.896,0.813
agebpop,,1.000,0.708,0.781,0.875,0.835
g.5km_idw10,,,1.000,0.843,0.838,0.596
g.5kmarea,,,,1.000,0.931,0.591
g.5kmpop,,,,,1.000,0.713
vor,,,,,,1.000
